In [1]:
%matplotlib inline

## Datasets Preparation

Check How many Images we have from each of the classes

In [10]:
import os

path = "/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_train_26-04-2024_15-44-44/ground_truth_trichomes_datasets/trichome_dataset_125_good_quality/train_test"

train_path = os.path.join(path, "train")
test_path = os.path.join(path, "val")

classes = os.listdir(train_path)
print("--- Train Images ---")
for class_name in classes:
    class_path = os.path.join(train_path, class_name)

    class_count = len(os.listdir(class_path))
    print(f"Class {class_name},  {class_count} images: ")
    print()
    
    
classes = os.listdir(test_path)
print("--- Val Images ---")
for class_name in classes:
    class_path = os.path.join(test_path, class_name)

    class_count = len(os.listdir(class_path))
    print(f"Class {class_name},  {class_count} images: ")
    print()


--- Train Images ---
Class cloudy,  533 images: 

Class amber,  371 images: 

Class clear,  202 images: 

--- Val Images ---
Class clear,  51 images: 

Class cloudy,  133 images: 

Class amber,  93 images: 



## Create Balanced Data set from the good quality images of the 1.25x 
- This dataset will have 200 images for each class.
- I want to measure the performance of balanced dataset compared to the original which received good score

In [9]:
import os
import shutil
import random

# Define paths
original_path = "/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_train_26-04-2024_15-44-44/ground_truth_trichomes_datasets/trichome_dataset_125_good_quality/train_test"
balanced_path = "/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_train_26-04-2024_15-44-44/ground_truth_trichomes_datasets/trichome_dataset_125_good_quality/train_test_balanced"

train_path = os.path.join(original_path, "train")
balanced_train_path = os.path.join(balanced_path, "train")

# Ensure the balanced dataset directories exist
os.makedirs(balanced_train_path, exist_ok=True)

def balance_class_images(source_class_path, target_class_path, target_count):
    """Balance the number of images for a class."""
    images = os.listdir(source_class_path)
    if len(images) > target_count:
        images = random.sample(images, target_count)  # Randomly select target_count images if there are too many
    else:
        # Duplicate images if there are fewer than target_count images
        images = images * (target_count // len(images)) + images[:target_count % len(images)]

    # Copy selected images to the target directory
    for i, img_name in enumerate(images):
        src_img_path = os.path.join(source_class_path, img_name)
        dest_img_path = os.path.join(target_class_path, f"{i}_{img_name}")
        shutil.copy(src_img_path, dest_img_path)

# Process train and val directories
for data_type, src_path, dest_path in [("train", train_path, balanced_train_path)]:
    classes = os.listdir(src_path)
    for class_name in classes:
        source_class_path = os.path.join(src_path, class_name)
        target_class_path = os.path.join(dest_path, class_name)
        os.makedirs(target_class_path, exist_ok=True)

        print(f"Balancing {data_type} set - Class {class_name} to 200 images.")
        balance_class_images(source_class_path, target_class_path, 200)

print("Balanced dataset created successfully!")



Balancing train set - Class cloudy to 200 images.
Balancing train set - Class amber to 200 images.
Balancing train set - Class clear to 200 images.
Balanced dataset created successfully!


- Split the Val dataset to Val and Test in order to eval the model

In [10]:
import os
import shutil
import random

# Define paths
original_val_path = "/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_train_26-04-2024_15-44-44/ground_truth_trichomes_datasets/trichome_dataset_125_good_quality/train_test/val"
split_val_path = "/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_train_26-04-2024_15-44-44/ground_truth_trichomes_datasets/trichome_dataset_125_good_quality/train_test_balanced/val"
split_test_path = "/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_train_26-04-2024_15-44-44/ground_truth_trichomes_datasets/trichome_dataset_125_good_quality/train_test_balanced/test"

# Ensure the split dataset directories exist
os.makedirs(split_val_path, exist_ok=True)
os.makedirs(split_test_path, exist_ok=True)

def split_images(source_class_path, val_class_path, test_class_path, test_ratio=0.2):
    """Split images into validation and test sets based on the specified test ratio."""
    images = os.listdir(source_class_path)
    random.shuffle(images)

    # Calculate the number of images for test and validation sets
    test_size = int(len(images) * test_ratio)
    test_images = images[:test_size]
    val_images = images[test_size:]

    # Copy test images
    for img_name in test_images:
        src_img_path = os.path.join(source_class_path, img_name)
        dest_img_path = os.path.join(test_class_path, img_name)
        shutil.copy(src_img_path, dest_img_path)

    # Copy validation images
    for img_name in val_images:
        src_img_path = os.path.join(source_class_path, img_name)
        dest_img_path = os.path.join(val_class_path, img_name)
        shutil.copy(src_img_path, dest_img_path)

# Process each class in the original validation set
classes = os.listdir(original_val_path)
for class_name in classes:
    source_class_path = os.path.join(original_val_path, class_name)
    val_class_path = os.path.join(split_val_path, class_name)
    test_class_path = os.path.join(split_test_path, class_name)

    # Create directories for each class in the split sets
    os.makedirs(val_class_path, exist_ok=True)
    os.makedirs(test_class_path, exist_ok=True)

    print(f"Splitting {class_name} into validation and test sets (80-20 split).")
    split_images(source_class_path, val_class_path, test_class_path, test_ratio=0.2)

print("Validation and test datasets created successfully!")


Splitting clear into validation and test sets (80-20 split).
Splitting cloudy into validation and test sets (80-20 split).
Splitting amber into validation and test sets (80-20 split).
Validation and test datasets created successfully!


In [11]:
# import YOLO model
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import os


data_path = "/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_train_26-04-2024_15-44-44/ground_truth_trichomes_datasets/trichome_dataset_125_good_quality/train_test_balanced"
test_data_path = "/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_train_26-04-2024_15-44-44/ground_truth_trichomes_datasets/trichome_dataset_125_good_quality/train_test_balanced/test"

## Comparison between Different Yolo Models
- YOLO v5
- YOLO v8
- YOLO v9
- YOLO v11



In [6]:
yolo_classification_checkpoints = {
    "YOLOv8": {
        "Nano": "yolov8n-cls.pt",
        "Small": "yolov8s-cls.pt",
        "Medium": "yolov8m-cls.pt",
        "Large": "yolov8l-cls.pt",
        "XLarge": "yolov8x-cls.pt",
    },
        "YOLOv11": {
        "Nano": "yolo11n-cls.pt",
        "Small": "yolo11s-cls.pt",
        "Medium": "yolo11m-cls.pt",
        "Large": "yolo11l-cls.pt",
        "XLarge": "yolo11x-cls.pt",
    },
}

Functions to fine tune each model and then evaluate it

In [8]:
def train_model(data_path, model_name, model_size, epochs=10, imgsz=128):
    # Load the model checkpoint
    checkpoint = yolo_classification_checkpoints[model_name][model_size]
    model = YOLO(checkpoint)
    
    # Train the model
    model.train(data=data_path, epochs=epochs, imgsz=imgsz)
    
    return model

def eval_model(model, test_data_path):
    # Get class names from the test dataset directory
    class_names = sorted(os.listdir(test_data_path))
    class_to_idx = {class_name: idx for idx, class_name in enumerate(class_names)}

    # Initialize lists for true labels (y_test) and predicted labels (y_pred)
    y_test = []
    y_pred = []

    # Initialize a list to store image data for plotting
    images_for_plotting = []

    # Process each class in the test directory
    for class_name in class_names:
        class_dir = os.path.join(test_data_path, class_name)
        for img_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_name)
            
            # Load the image and run the model prediction
            img = cv2.imread(img_path)
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB for matplotlib
            results = model(img)
            
            # Get the predicted class index
            predicted_idx = int(results[0].probs.data.argmax())
            
            # Append true and predicted labels
            y_test.append(class_to_idx[class_name])
            y_pred.append(predicted_idx)
            
            # Store the image, true label, and predicted label for plotting
            images_for_plotting.append((img_rgb, class_name, class_names[predicted_idx]))

    # Evaluate using classification metrics
    print("Classification Report:")
    print(classification_report(y_test, y_pred, target_names=class_names))

    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
    
def plot_results(images_for_plotting):
    # Plot images with true and predicted labels
    plt.figure(figsize=(12, 12))
    num_images = len(images_for_plotting)  # Display all images (adjust as needed)
    rows = (num_images // 4) + 1  # Adjust rows based on the number of images

    for i in range(num_images):
        img, true_label, pred_label = images_for_plotting[i]
        plt.subplot(rows, 4, i + 1)  # Dynamic row setting
        plt.imshow(img)
        plt.title(f"True: {true_label}\nPred: {pred_label}", color="green" if true_label == pred_label else "red")
        plt.axis('off')

    plt.tight_layout()
    plt.show()

In [12]:
# run the functions for each of the models
for model_name in yolo_classification_checkpoints:
    for model_size in yolo_classification_checkpoints[model_name]:
        print(f"Training {model_name} {model_size} model...")
        model = train_model(data_path, model_name, model_size, epochs=10, imgsz=128)
        
        print(f"Evaluating {model_name} {model_size} model...")
        eval_model(model, test_data_path)
        
        # print(f"Plotting results for {model_name} {model_size} model...")
        # plot_results(images_for_plotting)

Training YOLOv8 Nano model...
Ultralytics 8.3.27 🚀 Python-3.10.13 torch-2.1.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3090, 24161MiB)
engine/trainer: task=classify, mode=train, model=yolov8n-cls.pt, data=/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_train_26-04-2024_15-44-44/ground_truth_trichomes_datasets/trichome_dataset_125_good_quality/train_test_balanced, epochs=10, time=None, patience=100, batch=16, imgsz=128, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train11, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False,

train: Scanning /sise/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_train_26-04-2024_15-44-44/ground_truth_trichomes_datasets/trichome_dataset_125_good_quality/train_test_balanced/train... 600 images, 0 corrupt: 100%|██████████| 600/600 [00:00<?, ?it/s]
/home/etaylor/.conda/envs/yolo8/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 6, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
val: Scanning /sise/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_train_26-04-2024_15-44-44/ground_truth_trichomes_datasets/trichome_dataset_125_good_quality/train_test_balanced/val... 223 images, 0 corrupt

optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 26 weight(decay=0.0), 27 weight(decay=0.0005), 27 bias(decay=0.0)
Image sizes 128 train, 128 val
Using 8 dataloader workers
Logging results to /home/etaylor/runs/classify/train11
Starting training for 10 epochs...


/home/etaylor/.conda/envs/yolo8/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 6, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(



      Epoch    GPU_mem       loss  Instances       Size


...

                   all      0.937          1



10 epochs completed in 0.032 hours.
Optimizer stripped from /home/etaylor/runs/classify/train15/weights/last.pt, 112.5MB
Optimizer stripped from /home/etaylor/runs/classify/train15/weights/best.pt, 112.5MB

Validating /home/etaylor/runs/classify/train15/weights/best.pt...
Ultralytics 8.3.27 🚀 Python-3.10.13 torch-2.1.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3090, 24161MiB)
YOLOv8x-cls summary (fused): 133 layers, 56,127,043 parameters, 0 gradients, 153.8 GFLOPs
train: /sise/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_train_26-04-2024_15-44-44/ground_truth_trichomes_datasets/trichome_dataset_125_good_quality/train_test_balanced/train... found 600 images in 3 classes ✅ 
val: /sise/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_train_26-04-2024_15-44-44/ground_truth_trichomes_datasets/trichome_dataset_125_good_quality/train_test_balanced/val... found 223 images in 3 classes ✅ 
test: /sise/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_p

               classes   top1_acc   top5_acc: 100%|██████████| 7/7 [00:00<00:00, 13.19it/s]


                   all      0.951          1
Speed: 0.0ms preprocess, 2.3ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to /home/etaylor/runs/classify/train15
Evaluating YOLOv8 XLarge model...

0: 128x128 amber 1.00, cloudy 0.00, clear 0.00, 6.9ms
Speed: 1.8ms preprocess, 6.9ms inference, 0.1ms postprocess per image at shape (1, 3, 128, 128)

0: 128x128 amber 0.91, cloudy 0.08, clear 0.00, 7.0ms
Speed: 1.6ms preprocess, 7.0ms inference, 0.0ms postprocess per image at shape (1, 3, 128, 128)

0: 128x128 amber 0.98, cloudy 0.01, clear 0.00, 6.9ms
Speed: 1.4ms preprocess, 6.9ms inference, 0.0ms postprocess per image at shape (1, 3, 128, 128)

0: 128x128 amber 1.00, cloudy 0.00, clear 0.00, 7.1ms
Speed: 1.3ms preprocess, 7.1ms inference, 0.0ms postprocess per image at shape (1, 3, 128, 128)

0: 128x128 amber 0.88, cloudy 0.11, clear 0.02, 7.7ms
Speed: 1.7ms preprocess, 7.7ms inference, 0.0ms postprocess per image at shape (1, 3, 128, 128)

0: 128x128 amber 1.00, cloudy 

FileNotFoundError: [Errno 2] No such file or directory: 'yolov11n-cls.pt'